# Words to Floor Plan (W2FP) Generative adversarial network (GAN)

#### Imports
Packages and libraries used through-out the project.

In [1]:
import time
import json

import tensorflow as tf
import tensorflowjs as tfjs
import matplotlib.pyplot as plt
from tensorflow.keras import Model, layers, losses
from IPython import display

tf.__version__

'2.8.0'

#### Hyper-parametes

In [2]:
IMG_ROOT="./data/images/"

BATCH_SIZE=32
EPOCHS=50

MAX_SEQUENCE_LENGTH=128
VOCAB_SIZE = 128

#### Text vectorization
Defining text conversion utility, which processes text to be appropriate for neural network input.

In [3]:
int_vectorize_layer = layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

2022-03-20 14:55:26.614882: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-20 14:55:26.645558: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-20 14:55:26.645722: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-20 14:55:26.646423: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

#### Dataset loading

In [4]:
with open("data/labels.json") as file:
    labels_json = json.load(file)

filelist = []
labels_array = []

for key, value in labels_json.items():
    filelist.append(key)
    labels_array.append(value)

labels_dataset = tf.convert_to_tensor(labels_array)
int_vectorize_layer.adapt(labels_dataset)

In [5]:
images_array = []
for name in filelist:
    image = Image.open(IMG_ROOT + name)
    image_w, image_h = image.size

    padded_image = Image.new(image.mode, (64, 64), (255, 255, 255))
    offset = ((64 - image_w) // 2, (64 - image_h) // 2)
    padded_image.paste(image, offset)

    flat_image = []

    for y in range(64):
        for x in range(64):
            r, g, b = padded_image.getpixel((x, y))
            if r == 0 and g == 0 and b == 0:
                flat_image.append(-1)
            elif r == 255 and g == 0 and b == 0:
                flat_image.append(-2/3)
            elif r == 255 and g == 232 and b == 0:
                flat_image.append(-1/3)
            elif r == 0 and g == 0 and b == 255:
                flat_image.append(0)
            elif r == 0 and g == 255 and b == 0:
                flat_image.append(1/3)
            elif r == 0 and g == 255 and b == 255:
                flat_image.append(2/3)
            elif r == 255 and g == 255 and b == 255:
                flat_image.append(1)

    flat_image = np.array(flat_image).reshape(64, 64, 1)
    images_array.append(flat_image)


images_dataset = tf.convert_to_tensor(images_array)

#### Generator neural network

In [6]:
def create_generator_model() -> Model:
    model = tf.keras.Sequential()
    model.add(int_vectorize_layer)

    # `VOCAB_SIZE + 1` since `0` is used additionally for padding.
    model.add(layers.Embedding(VOCAB_SIZE + 1, 128, input_length=MAX_SEQUENCE_LENGTH, mask_zero=True)),
    model.add(layers.Conv1D(64, 5, padding="same", activation="relu", strides=2))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(8*8*64))

    model.add(layers.Reshape((8, 8, 64)))

    model.add(layers.Conv2DTranspose(512, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(256, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

    return model

In [7]:
generator = create_generator_model()
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 128, 128)          16512     
                                                                 
 conv1d (Conv1D)             (None, 64, 64)            41024     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 4096)              266240    
                                                                 
 reshape (Reshape)           (None, 8, 8, 64)          0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 8, 8, 512)        819200    
 nspose)                                                

#### Discriminator neural network

In [7]:
def create_discriminator_model() -> Model:
    model = tf.keras.Sequential()

    model.add(
        layers.Conv2D(64, (4, 4), strides=(2, 2), padding='same', use_bias=False, input_shape=[64, 64, 1]))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(
        layers.Conv2D(128, (4, 4), strides=(2, 2), padding='same', use_bias=False, input_shape=[64, 32, 32]))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(
        layers.Conv2D(256, (4, 4), strides=(2, 2), padding='same', use_bias=False, input_shape=[128, 16, 16]))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(
        layers.Conv2D(512, (4, 4), strides=(2, 2), padding='same', use_bias=False, input_shape=[256, 8, 8]))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(
        layers.Conv2D(512, (4, 4), strides=(1, 1), padding='same', use_bias=False, input_shape=[512, 4, 4]))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [8]:
discriminator = create_discriminator_model()
discriminator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        1024      
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 64)       256       
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 32, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 128)       131072    
                                                                 
 batch_normalization_1 (Batc  (None, 16, 16, 128)      512       
 hNormalization)                                                 
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 16, 16, 128)       0

#### Loss functions

In [9]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

#### Optimizer functions

In [10]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

#### Training loop

In [11]:
@tf.function
def train_step(images):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(labels_dataset, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [12]:
def train(image_dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for batch in image_dataset:
      train_step(batch)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [ ]:
train(images_dataset, EPOCHS)

#### Model saving

In [19]:
# generator.save('models/tf')
tfjs.converters.save_keras_model(generator, 'models/tfjs')

NotImplementedError: Save or restore weights that is not an instance of `tf.Variable` is not supported in h5, use `save_format='tf'` instead. Received a model or layer TextVectorization with weights [<keras.layers.preprocessing.index_lookup.VocabWeightHandler object at 0x7f17a0355c90>]